<a href="https://colab.research.google.com/github/aetev/Learning-stuff-/blob/main/DENOISED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pandas_ta

In [ ]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import tensorflow as tf
import matplotlib.pyplot as plt
from numpy.lib.stride_tricks import sliding_window_view
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, LSTM

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/eurusd_hour.csv')
close = df['AC']
open = df['AO']
high = df['AH']
low = df['AL']
change = df['ACh']
print(df)

In [ ]:
SMA = ta.sma(close,4,offset=-2)
NOISE = close-SMA
BAR_SIZE = high-low
#OPEN_SIZE =

plt.xlim(20000,20100)
plt.ylim(1.56,1.6)
plt.plot(SMA)
plt.plot(close)
plt.show()
plt.xlim(20000,20100)
plt.plot(change)
plt.plot(NOISE)
plt.show()

In [ ]:
# Define the window length and stride
window_length = 4
stride = 1

BAR_SIZED = pd.Series(BAR_SIZE,name='bar_size')
changed = pd.Series(change,name='change')
NOISED = pd.Series(NOISE,name='noise')
combined_series = pd.concat([changed, NOISED,BAR_SIZED], axis=1)
combined_series = combined_series.dropna(axis=0)
print(combined_series)
noise = combined_series.pop('noise')
changed = combined_series.pop('change')
BAR_SIZED = combined_series.pop('bar_size')

# Create the sliding window view for time_series_1
changed = sliding_window_view(changed, window_length)[::stride]
BAR_SIZED = sliding_window_view(BAR_SIZED, window_length)[::stride]
NOISED = np.roll(noise, -window_length+1)
NOISED = NOISED[:changed.shape[0]]

BAR_SIZED = BAR_SIZED-np.mean(BAR_SIZED)/np.std(BAR_SIZED)
changed = changed/np.std(changed)
NOISED = NOISED/np.std(NOISED)

print(BAR_SIZED)

In [ ]:

print(changed[:, -1])
print(NOISED)


plt.xlim(20000,20100)
plt.plot(changed[:, -1])
plt.plot(NOISED)


In [ ]:
def activity_regularizer(multiplier, decay=0.99):
    # Initialize moving averages for mean and variance
    mean_avg = tf.Variable(0.0, trainable=False, name='mean_avg')
    variance_avg = tf.Variable(1.0, trainable=False, name='variance_avg')

    def regularizer(y_pred):
        # Calculate the current mean and variance
        current_mean = tf.reduce_mean(y_pred)
        current_variance = tf.math.reduce_variance(y_pred)

        # Update moving averages
        update_mean = tf.compat.v1.assign(mean_avg, decay * mean_avg + (1 - decay) * current_mean)
        update_variance = tf.compat.v1.assign(variance_avg, decay * variance_avg + (1 - decay) * current_variance)

        with tf.control_dependencies([update_mean, update_variance]):
            # Use the moving averages for regularization
            mean = mean_avg
            variance = variance_avg

        return multiplier * (tf.square(mean - 0) + tf.square(variance - 1))

    return regularizer

In [ ]:
reg = activity_regularizer(.01)

# Create a sequential model
model = Sequential()


model.add(Dense(12, activation='gelu',activity_regularizer=reg))
model.add(Dense(12, activation='gelu',activity_regularizer=reg))
model.add(Dense(1, activation=None))

# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [ ]:
X_train=np.concatenate((changed, BAR_SIZED), axis=1)
y_train=NOISED

model.fit(X_train, y_train, epochs=10, batch_size=32,validation_split=.2)

In [ ]:
prediction = model.predict(X_train)

In [ ]:
cumsum = np.cumsum(np.cumsum(changed[:, -1]))

In [ ]:
plt.xlim(30000,30100)
plt.plot(cumsum+prediction)
plt.plot(cumsum)

In [ ]:
# Set the print options
np.set_printoptions(suppress=True, precision=4)
print(model.get_weights())